# Connecting to an instance of GroundX On-Prem
This notebook assumes you have set up a GroundX On-Prem Instance
https://github.com/eyelevelai/eyelevel-iac

This notebook uses a GroundX On-Prem instance to do parsing and retreival, then passes retreived context to a local LLM, thus doing RAG within a self-contained environment.

Recommended to use an A100 with High-RAM on Google Colab.


In [ ]:
!pip install groundx-python-sdk==1.3.29

Call `kubectl -n eyelevel get svc` to get the external_ip

In [ ]:
from groundx import Groundx
from groundx.configuration import Configuration

#replace with your external IP
external_ip = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx-xxxxxxxxxx.us-east-2.elb.amazonaws.com'

groundx = Groundx(
    configuration=Configuration(
        host=f"http://{external_ip}/api",
        #replace with your API Key
        api_key="xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx",
    )
)

# uploading

In [ ]:
"""Setting Up Bucket
"""

import json
response = groundx.buckets.create(
    name="demo_bucket"
)
print(response.body)
bucekt_id = json.loads(response.body)['bucket']['bucketId']

{
 "bucket": {
  "bucketId": 2,
  "name": "demo_bucket"
 }
}



In [ ]:
"""Uploading Document to Bucket
"""

response = groundx.documents.ingest_local([{
    "blob": open("document.pdf", "rb"),
    "metadata": {
        "bucketId": bucekt_id,
        "fileName": "document.pdf",
        "fileType": "pdf"
    }
}])
print(response.body)
processId = json.loads(response.body)['ingest']['processId']

{
 "ingest": {
  "processId": "a4c236c0-6294-416b-8a84-755bb26d1f01",
  "status": "queued"
 }
}



In [ ]:
processId

'a4c236c0-6294-416b-8a84-755bb26d1f01'

In [ ]:
"""Checking Status
wait for it to "status":"complete" before progressing
"""

from groundx import Groundx

response = groundx.documents.get_processing_status_by_id(
    process_id=processId
)
print(json.loads(response.body)['ingest']['status'])
print('---')
print(response.body)

complete
---
{
 "ingest": {
  "processId": "a4c236c0-6294-416b-8a84-755bb26d1f01",
  "progress": {
   "complete": {
    "documents": [
     {
      "documentId": "de4a04ca-1db2-42dc-9930-8ff52fe75bd5",
      "fileName": "document.pdf",
      "fileSize": "136 KB",
      "fileTokens": 35211,
      "fileType": "pdf",
      "bucketId": 2,
      "processId": "a4c236c0-6294-416b-8a84-755bb26d1f01",
      "sourceUrl": "http://minio.eyelevel.svc.cluster.local/eyelevel/prod/file/758a83e3-e1bc-40b5-aa37-a97ec0210eee/document.pdf",
      "status": "complete",
      "xrayUrl": "http://minio.eyelevel.svc.cluster.local/eyelevel/layout/processed/a4c236c0-6294-416b-8a84-755bb26d1f01/de4a04ca-1db2-42dc-9930-8ff52fe75bd5-xray.json"
     }
    ],
    "total": 1
   }
  },
  "status": "complete"
 }
}



# Retreival

In [ ]:
import json

query = "What testing protocol should I follow if the package is shipping LTL?"

response = groundx.search.content(
    id=bucekt_id,
    query=query
)
context = json.loads(response.body)['search']['text']

In [ ]:
print(context)

The following text excerpts are from the same section of a document named 'document.pdf':

Text excerpt from page number 1:
The JSONL object for the image is as follows:
```json
{
  "figure_title": "Walmart Package Testing Flowchart",
  "figure_number": 1,
  "description": "This flowchart outlines the process for testing and labeling packages for Walmart's private brands.",
  "components": [
    {
      "name": "New or existing item",
      "description": "Determine if the item is new or already exists."
    },
    {
      "name": "Is Item required to be SIOC",
      "description": "Check if the item must be shipped as SIOC."
    },
    {
      "name": "Does the item ship LTL from the Fulfillment Center?",
      "description": "Determine if the item should be shipped via LTL."
    },
    {
      "name": "Follow ISTA 3B protocol",
      "description": "Follow specific ISTA 3B protocol if applicable."
    },
    {
      "name": "Follow ISTA 6A Fed-Ex International protocol",
      "descr

# Making an Inference with an LLM Locally

In [ ]:
import os
from huggingface_hub import login
from google.colab import userdata

login(userdata.get('HuggingFace'))

In [ ]:
from transformers import pipeline
pipe = pipeline("text-generation", model="meta-llama/Llama-3.2-3B-Instruct", max_length = 6000, device='cuda')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

messages = [
    {"role": "user", "content": f"use this information to answer my next query\n\n{context}"},
    {"role": "user", "content": f'{query} Think about your answer then generate a response'},
]
print(pipe(messages)[0]['generated_text'][-1]['content'])

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Based on the provided text excerpts, if the package is shipping LTL (Less Than Truckload), the testing protocol to follow is the ISTA 3B protocol. This is because the flowchart indicates that if the item ships LTL from the Fulfillment Center, the user should follow the ISTA 3B protocol.

Additionally, the text excerpt from page 3 states that if the item does not meet the conditions for shipping SIOC, it will be shipped via LTL. Therefore, if the package is shipping LTL, it is likely that the ISTA 3B protocol should be followed.

It's worth noting that the text excerpt from page 2 also mentions that if any of the conditions are met, the item will ship LTL. These conditions include:

* >150lbs
* >108" in longest dimension (9ft)
* >165" in length (L) + girth (2W+2H)

If any of these conditions are met, the package will ship LTL and the ISTA 3B protocol should be followed.
